In [1]:
pip install transformers==4.5.0

     |████████████████████████████████| 2.1 MB 5.3 MB/s 
     |████████████████████████████████| 3.3 MB 25.6 MB/s 
     |████████████████████████████████| 895 kB 47.8 MB/s 


In [2]:
import numpy as np
import pandas as pd
import torch
from transformers import RobertaTokenizer, RobertaModel, AdamW, get_linear_schedule_with_warmup
from torch.utils.data import Dataset, DataLoader,TensorDataset
from tqdm.auto import tqdm
import torch.nn as nn
import torch.nn.functional as F
from sklearn import metrics
from sklearn.metrics import accuracy_score,f1_score,recall_score
import random
from argparse import Namespace
from torch.autograd import Variable
device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
def set_seed(seed):
    random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
set_seed(0)

In [4]:
args = Namespace(
    num_class = 3,
    num_epochs = 3,
    lr = 5e-5,
    batch_size = 32,
    save_dict_path = './bertbasline/'
)

In [5]:
tokenizer =  RobertaTokenizer.from_pretrained('roberta-base')

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [6]:
class Model(nn.Module):
    def __init__(self, args):
        super(Model,self).__init__()
        self.hidden_size = 768
        self.num_class = args.num_class

        self.bert = RobertaModel.from_pretrained('roberta-base')
        self.fc = nn.Linear(self.hidden_size, self.num_class)                 
        
        self.dropout = nn.Dropout(0.1)

    def forward(self, x):
        outputs = self.bert(input_ids=x[0], attention_mask=x[1])
        pooler_out = outputs[1]
        out = self.dropout(pooler_out)
        out = self.fc(out)
        return out
model1 = Model(args).to(device)
model2 = Model(args).to(device)

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/501M [00:00<?, ?B/s]

In [7]:
def data_process(filename,label):
    df = pd.read_excel(filename)
    df = df[['content',label]]
    df = df.dropna()
    df.columns=['content', 'label']
    print(df['label'].value_counts())
    return df

In [ ]:
#sep
def data_process(filename,label):
    df = pd.read_excel(filename)
    df = df[['content','topic',label]]
    df = df.dropna()
    df['content_mix'] = df['topic'] + '</s> ' + df['content']
    df = df[['content_mix',label]]
    df.columns=['content', 'label']
    print(df['label'].value_counts())
    return df

In [8]:
df_train1 = data_process("/content/drive/MyDrive/冒犯/data/train_set.xlsx",'label_1')
df_val1 = data_process("/content/drive/MyDrive/冒犯/data/val_set.xlsx",'label_1')
df_test1 = data_process("/content/drive/MyDrive/冒犯/data/test_set.xlsx",'label_1')
df_train2 = data_process("/content/drive/MyDrive/冒犯/data/train_set.xlsx",'label_2')
df_val2 = data_process("/content/drive/MyDrive/冒犯/data/val_set.xlsx",'label_2')
df_test2 = data_process("/content/drive/MyDrive/冒犯/data/test_set.xlsx",'label_2')

0    7200
1     692
2     482
Name: label, dtype: int64
0    897
1     87
2     63
Name: label, dtype: int64
0    906
1     85
2     56
Name: label, dtype: int64
0    7202
1     662
2     510
Name: label, dtype: int64
0    904
1     82
2     61
Name: label, dtype: int64
0    894
1     85
2     68
Name: label, dtype: int64


In [9]:
def myDataset(data_set):
  texts = data_set['content'].tolist()
  labels = data_set['label'].tolist()
  input_ids = []
  attention_masks = []
  for text in texts:
      encoded_dict = tokenizer.encode_plus(
                    text,                     
                    add_special_tokens = True,
                    max_length = 256,        
                    pad_to_max_length = True,
                    return_attention_mask = True,
                    return_tensors = 'pt',   
      )
      
     
      input_ids.append(encoded_dict['input_ids'])
      attention_masks.append(encoded_dict['attention_mask'])


  input_ids = torch.cat(input_ids, dim=0)
  attention_masks = torch.cat(attention_masks, dim=0)
  labels = torch.tensor(labels)

  dataset = TensorDataset(input_ids, attention_masks, labels)

  return dataset

In [ ]:
class MultiCEFocalLoss(torch.nn.Module):
    def __init__(self, class_num, gamma=2, alpha=None, reduction='mean'):
        super(MultiCEFocalLoss, self).__init__()
        if alpha is None:
            self.alpha = Variable(torch.ones(class_num, 1))
        else:
            self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction
        self.class_num =  class_num

    def forward(self, predict, target):
        pt = F.softmax(predict, dim=1) # softmmax获取预测概率
        ####
        class_mask = F.one_hot(target, self.class_num) #获取target的one hot编码
        ####
        #ids = target.view(-1, 1) 
        #alpha = self.alpha[ids.data.view(-1)].view(-1,1) 
        # 注意，这里的alpha是给定的一个list(tensor),里面的元素分别是每一个类的权重因子
        probs = (pt * class_mask).sum(1).view(-1, 1) # 利用onehot作为mask，提取对应的pt
        log_p = probs.log()
# 同样，原始ce上增加一个动态权重衰减因子
        loss = -(torch.pow((1 - probs), self.gamma)) * log_p

        if self.reduction == 'mean':
            loss = loss.mean()
        elif self.reduction == 'sum':
            loss = loss.sum()
        return loss
myloss = MultiCEFocalLoss(class_num = 3).to(device)

In [ ]:
def load_data(args,df_train,df_val,df_test):

    train_dataset = myDataset(df_train)
    dev_dataset = myDataset(df_val)
    test_dataset = myDataset(df_test)
    # 计算机内存充足时pin_memory = True，可加快计算速度
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    dev_loader = DataLoader(dev_dataset, batch_size=32, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    return train_loader, dev_loader,test_loader

In [ ]:
train_loader1,dev_loader1,test_loader1 = load_data(args,df_train1,df_val1,df_test1)
train_loader2,dev_loader2,test_loader2 = load_data(args,df_train2,df_val2,df_test2)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2079: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
optimizer1 = AdamW(model1.parameters(), lr=args.lr, eps=1e-8)
optimizer2 = AdamW(model2.parameters(), lr=args.lr, eps=1e-8)

In [ ]:
def train(args, model, optimizer, train_loader, dev_loader):
    
    num_epochs = 4   # bert建议是3，但使用bert微调，结果不稳定，适当增大epochs
    total_steps = len(train_loader)* num_epochs
    logging_step = 50
    validation = True
    learning_rate = args.lr
    scheduler = get_linear_schedule_with_warmup(
        optimizer, 
        num_warmup_steps = 0,
        num_training_steps = total_steps,
        )  # 确实有效果，尝试其他warmup
    for epoch in range(num_epochs):
        model.train()
      
        step = 1
        train_loss = train_acc = 0

        for data in tqdm(train_loader):

            data = [i.to(device) for i in data]
            # 模型输入: input_ids,  attention_mask
            # 模型输出: logits,
            output = model(data) #为啥可以直接输入
            labels = data[2]
            optimizer.zero_grad()

            if epoch >= 2:
              loss = myloss(output, labels)
            else:
              loss = F.cross_entropy(output, labels)
            train_loss += loss.item()
            loss.backward()
            
            pred_class = torch.max(output, 1)[1]
            train_acc += (pred_class == labels).float().mean()

            nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            scheduler.step() 

            step += 1
            # optimizer.param_groups[0]["lr"] -= learning_rate / (total_steps)
            if step % logging_step == 0:

                print(
                    f"Epoch {epoch + 1} | Step {step} | Train_loss = {train_loss / logging_step:.3f}, Train_acc = {train_acc / logging_step:.3f}"
                )
                train_loss = train_acc = 0
  
        if validation:
            print("Evaluating Dev Set ...")
            model.eval()
            dev_loss = 0
            predict_all = np.array([], dtype=int)
            labels_all = np.array([], dtype=int)
            with torch.no_grad():
    
                for data in tqdm(dev_loader):
                    data = [i.to(device) for i in data]
                    output = model(data)
                    labels = data[2]
                    loss = F.cross_entropy(output, labels)
                    dev_loss += loss.item()

                    labels = data[2].data.cpu().numpy()
                    pred_class = torch.max(output.data, 1)[1].cpu().numpy()
                    
                    labels_all = np.append(labels_all, labels)
                    predict_all = np.append(predict_all, pred_class)

            dev_acc = metrics.accuracy_score(labels_all, predict_all)

            f1_score_micro = metrics.f1_score(labels_all, predict_all, average='micro')
            f1_score_macro = metrics.f1_score(labels_all, predict_all, average='macro')

            report = metrics.classification_report(labels_all, predict_all, digits=3)
            confusion = metrics.confusion_matrix(labels_all, predict_all)
            print(f"Validation | Epoch {epoch + 1} | loss = {dev_loss / len(dev_loader):.3f} | acc = {dev_acc:.3f}")
            print(f"F1 Score (Micro) = {f1_score_micro}")
            print(f"F1 Score (Macro) = {f1_score_macro}")
            print(report)
            print(confusion)

In [ ]:
train(args, model1, optimizer1, train_loader1, dev_loader1)

  0%|          | 0/262 [00:00<?, ?it/s]

Epoch 1 | Step 50 | Train_loss = 0.490, Train_acc = 0.850
Epoch 1 | Step 100 | Train_loss = 0.473, Train_acc = 0.844
Epoch 1 | Step 150 | Train_loss = 0.383, Train_acc = 0.872
Epoch 1 | Step 200 | Train_loss = 0.425, Train_acc = 0.861
Epoch 1 | Step 250 | Train_loss = 0.404, Train_acc = 0.854
Evaluating Dev Set ...


  0%|          | 0/33 [00:00<?, ?it/s]

Validation | Epoch 1 | loss = 0.414 | acc = 0.857
F1 Score (Micro) = 0.8567335243553008
F1 Score (Macro) = 0.30761316872427985
              precision    recall  f1-score   support

           0      0.857     1.000     0.923       897
           1      0.000     0.000     0.000        87
           2      0.000     0.000     0.000        63

    accuracy                          0.857      1047
   macro avg      0.286     0.333     0.308      1047
weighted avg      0.734     0.857     0.791      1047

[[897   0   0]
 [ 87   0   0]
 [ 63   0   0]]


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


...

Validation | Epoch 3 | loss = 0.399 | acc = 0.855
F1 Score (Micro) = 0.8548233046800382
F1 Score (Macro) = 0.5819298369242875
              precision    recall  f1-score   support

           0      0.928     0.936     0.932       897
           1      0.269     0.287     0.278        87
           2      0.612     0.476     0.536        63

    accuracy                          0.855      1047
   macro avg      0.603     0.567     0.582      1047
weighted avg      0.854     0.855     0.854      1047

[[840  52   5]
 [ 48  25  14]
 [ 17  16  30]]


  0%|          | 0/262 [00:00<?, ?it/s]

Epoch 4 | Step 50 | Train_loss = 0.044, Train_acc = 0.934
Epoch 4 | Step 100 | Train_loss = 0.059, Train_acc = 0.943
Epoch 4 | Step 150 | Train_loss = 0.064, Train_acc = 0.936
Epoch 4 | Step 200 | Train_loss = 0.047, Train_acc = 0.946
Epoch 4 | Step 250 | Train_loss = 0.043, Train_acc = 0.956
Evaluating Dev Set ...


  0%|          | 0/33 [00:00<?, ?it/s]

Validation | Epoch 4 | loss = 0.414 | acc = 0.850
F1 Score (Micro) = 0.8500477554918816
F1 Score (Macro) = 0.5668026920471048
              precision    recall  f1-score   support

           0      0.928     0.933     0.931       897
           1      0.277     0.356     0.312        87
           2      0.667     0.349     0.458        63

    accuracy                          0.850      1047
   macro avg      0.624     0.546     0.567      1047
weighted avg      0.858     0.850     0.851      1047

[[837  56   4]
 [ 49  31   7]
 [ 16  25  22]]


In [ ]:
train(args, model2, optimizer2, train_loader2, dev_loader2)

  0%|          | 0/262 [00:00<?, ?it/s]

Epoch 1 | Step 50 | Train_loss = 0.502, Train_acc = 0.821
Epoch 1 | Step 100 | Train_loss = 0.434, Train_acc = 0.861
Epoch 1 | Step 150 | Train_loss = 0.427, Train_acc = 0.866
Epoch 1 | Step 200 | Train_loss = 0.414, Train_acc = 0.858
Epoch 1 | Step 250 | Train_loss = 0.380, Train_acc = 0.871
Evaluating Dev Set ...


  0%|          | 0/33 [00:00<?, ?it/s]

Validation | Epoch 1 | loss = 0.381 | acc = 0.860
F1 Score (Micro) = 0.8595988538681948
F1 Score (Macro) = 0.4686586703021358
              precision    recall  f1-score   support

           0      0.920     0.950     0.935       904
           1      0.000     0.000     0.000        82
           2      0.363     0.672     0.471        61

    accuracy                          0.860      1047
   macro avg      0.428     0.541     0.469      1047
weighted avg      0.815     0.860     0.835      1047

[[859   0  45]
 [ 55   0  27]
 [ 20   0  41]]


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


...

Validation | Epoch 3 | loss = 0.423 | acc = 0.858
F1 Score (Micro) = 0.8576886341929322
F1 Score (Macro) = 0.5276082736016976
              precision    recall  f1-score   support

           0      0.927     0.946     0.936       904
           1      0.287     0.354     0.317        82
           2      0.583     0.230     0.329        61

    accuracy                          0.858      1047
   macro avg      0.599     0.510     0.528      1047
weighted avg      0.857     0.858     0.853      1047

[[855  47   2]
 [ 45  29   8]
 [ 22  25  14]]


  0%|          | 0/262 [00:00<?, ?it/s]

Epoch 4 | Step 50 | Train_loss = 0.051, Train_acc = 0.922
Epoch 4 | Step 100 | Train_loss = 0.051, Train_acc = 0.941
Epoch 4 | Step 150 | Train_loss = 0.054, Train_acc = 0.951
Epoch 4 | Step 200 | Train_loss = 0.050, Train_acc = 0.944
Epoch 4 | Step 250 | Train_loss = 0.046, Train_acc = 0.949
Evaluating Dev Set ...


  0%|          | 0/33 [00:00<?, ?it/s]

Validation | Epoch 4 | loss = 0.446 | acc = 0.852
F1 Score (Micro) = 0.8519579751671442
F1 Score (Macro) = 0.5363235363091859
              precision    recall  f1-score   support

           0      0.929     0.936     0.932       904
           1      0.283     0.341     0.309        82
           2      0.486     0.295     0.367        61

    accuracy                          0.852      1047
   macro avg      0.566     0.524     0.536      1047
weighted avg      0.852     0.852     0.851      1047

[[846  49   9]
 [ 44  28  10]
 [ 21  22  18]]


In [ ]:
torch.save(model1.state_dict(),'/content/drive/MyDrive/model_parameter/f_model1_state_dict.pt')

In [ ]:
torch.save(model2.state_dict(),'/content/drive/MyDrive/model_parameter/f_model2_state_dict.pt')

In [ ]:
model1.load_state_dict(torch.load('/content/drive/MyDrive/model_parameter/f_model1_state_dict.pt'))
model2.load_state_dict(torch.load('/content/drive/MyDrive/model_parameter/f_model2_state_dict.pt'))

<All keys matched successfully>

In [ ]:
def test(model,test_loader):
    result = []
    #model.load_state_dict(torch.load('/content/drive/MyDrive/bertbasline/model_state_dict.pt'))
    model.eval()
    with torch.no_grad():
        for data in tqdm(test_loader):
            data = [i.to(device) for i in data]
            output = model(data)
            pred_class = torch.max(output, dim=1)[1]
            result += (pred_class.cpu().numpy().tolist())
    return np.array(result)

In [ ]:
def test_final(model1,model2,test_loader,other):
    res = []
    pred1 = test(model1,test_loader)
    pred2 = test(model2,test_loader)
    for i in range(len(pred1)):
      if pred1[i] == 0 and pred2[i] == 0:
        res.append(0)
      elif pred1[i] == 1 and pred2[i] == 1:
        res.append(2)
      elif pred1[i] == 2 and pred2[i] == 2: 
        res.append(4)
      elif (pred1[i] == 0 and pred2[i] == 1) or (pred1[i] == 1 and pred2[i] == 0) : 
        res.append(1)
      elif (pred1[i] == 1 and pred2[i] == 2) or (pred1[i] == 2 and pred2[i] == 1) : 
        res.append(3)
      else:
        res.append(other)
    return np.array(res)

In [ ]:
prd_final4 = test_final(model1,model2,test_loader1,4)
prd_final3 = test_final(model1,model2,test_loader1,3)
prd_final2 = test_final(model1,model2,test_loader1,2)
prd_final1 = test_final(model1,model2,test_loader1,1)
prd_final0 = test_final(model1,model2,test_loader1,0)

  0%|          | 0/33 [00:00<?, ?it/s]

  0%|          | 0/33 [00:00<?, ?it/s]

  0%|          | 0/33 [00:00<?, ?it/s]

  0%|          | 0/33 [00:00<?, ?it/s]

  0%|          | 0/33 [00:00<?, ?it/s]

  0%|          | 0/33 [00:00<?, ?it/s]

  0%|          | 0/33 [00:00<?, ?it/s]

  0%|          | 0/33 [00:00<?, ?it/s]

  0%|          | 0/33 [00:00<?, ?it/s]

  0%|          | 0/33 [00:00<?, ?it/s]

In [ ]:
df_test = data_process("/content/drive/MyDrive/冒犯/data/test_set.xlsx",'label')
test_dataset = myDataset(df_test)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
label_true = np.array(df_test['label'].to_list())

0    853
1     94
3     46
4     39
2     15
Name: label, dtype: int64


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2079: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
def result_show(label_true, pred):
  confusion = metrics.confusion_matrix(label_true, pred)
  report = metrics.classification_report(label_true, pred, digits=3)
  print(confusion)
  print(report)

In [ ]:
result_show(label_true, prd_final0)

[[789  40  16   4   4]
 [ 58  19  12   3   2]
 [  4   4   5   0   2]
 [ 12   4   8  10  12]
 [  8   5   6   6  14]]
              precision    recall  f1-score   support

           0      0.906     0.925     0.915       853
           1      0.264     0.202     0.229        94
           2      0.106     0.333     0.161        15
           3      0.435     0.217     0.290        46
           4      0.412     0.359     0.384        39

    accuracy                          0.799      1047
   macro avg      0.425     0.407     0.396      1047
weighted avg      0.798     0.799     0.796      1047



In [ ]:
result_show(label_true, prd_final1)

[[788  41  16   4   4]
 [ 58  19  12   3   2]
 [  4   4   5   0   2]
 [  8   8   8  10  12]
 [  8   5   6   6  14]]
              precision    recall  f1-score   support

           0      0.910     0.924     0.917       853
           1      0.247     0.202     0.222        94
           2      0.106     0.333     0.161        15
           3      0.435     0.217     0.290        46
           4      0.412     0.359     0.384        39

    accuracy                          0.798      1047
   macro avg      0.422     0.407     0.395      1047
weighted avg      0.799     0.798     0.796      1047



In [ ]:
result_show(label_true, prd_final2)

[[788  40  17   4   4]
 [ 58  19  12   3   2]
 [  4   4   5   0   2]
 [  8   4  12  10  12]
 [  8   5   6   6  14]]
              precision    recall  f1-score   support

           0      0.910     0.924     0.917       853
           1      0.264     0.202     0.229        94
           2      0.096     0.333     0.149        15
           3      0.435     0.217     0.290        46
           4      0.412     0.359     0.384        39

    accuracy                          0.798      1047
   macro avg      0.423     0.407     0.394      1047
weighted avg      0.801     0.798     0.797      1047



In [ ]:
result_show(label_true, prd_final3)

[[788  40  16   5   4]
 [ 58  19  12   3   2]
 [  4   4   5   0   2]
 [  8   4   8  14  12]
 [  8   5   6   6  14]]
              precision    recall  f1-score   support

           0      0.910     0.924     0.917       853
           1      0.264     0.202     0.229        94
           2      0.106     0.333     0.161        15
           3      0.500     0.304     0.378        46
           4      0.412     0.359     0.384        39

    accuracy                          0.802      1047
   macro avg      0.438     0.425     0.414      1047
weighted avg      0.804     0.802     0.801      1047



In [ ]:
result_show(label_true, prd_final4)

[[788  40  16   4   5]
 [ 58  19  12   3   2]
 [  4   4   5   0   2]
 [  8   4   8  10  16]
 [  8   5   6   6  14]]
              precision    recall  f1-score   support

           0      0.910     0.924     0.917       853
           1      0.264     0.202     0.229        94
           2      0.106     0.333     0.161        15
           3      0.435     0.217     0.290        46
           4      0.359     0.359     0.359        39

    accuracy                          0.798      1047
   macro avg      0.415     0.407     0.391      1047
weighted avg      0.799     0.798     0.796      1047

